In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1691628408595_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_orders').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bucket_refined = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/004_refined'.format(dia=_DIA_)
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
output_refined = 't_indicadores_tipos_credito'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_bureau_20230805')
bureau_00=sqlContext.read.format("parquet").load(path_lake)
bureau_00.registerTempTable("bureau_00")
bureau_00.cache()
bureau_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1716428

In [9]:
bureau_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ts_proc: long (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc_partition: long (nullable = true)

In [16]:
indicadores_00 = spark.sql(     
    """
        SELECT
            CREDIT_TYPE,
            CREDIT_ACTIVE,
            AMT_CREDIT_SUM
        FROM
            bureau_00

    """)
indicadores_00.registerTempTable("indicadores_00")
indicadores_00.cache()
indicadores_00.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1716428

In [17]:
indicadores_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+--------------+
|    CREDIT_TYPE|CREDIT_ACTIVE|AMT_CREDIT_SUM|
+---------------+-------------+--------------+
|Consumer credit|       Closed|       91323.0|
|    Credit card|       Active|      225000.0|
|Consumer credit|       Active|      464323.5|
|    Credit card|       Active|       90000.0|
|Consumer credit|       Active|     2700000.0|
|    Credit card|       Active|      180000.0|
|Consumer credit|       Active|       42103.8|
|Consumer credit|       Closed|      76878.45|
|Consumer credit|       Closed|      103007.7|
|    Credit card|       Active|        4500.0|
|    Credit card|       Active|      337500.0|
|Consumer credit|       Closed|      675000.0|
|       Mortgage|       Active|     7033500.0|
|Consumer credit|       Active|       89910.0|
|    Credit card|       Closed|           0.0|
|Consumer credit|       Closed|       48555.0|
|Consumer credit|       Closed|      135000.0|
|Consumer credit|       Active|      450000.0|
|    Credit c

In [18]:
tabela = output_refined

path_athena = os.path.join('s3://',bucket_refined,tabela)
print('path_tabela:', path_athena)
indicadores_00.coalesce(1).write.\
format("parquet").mode("overwrite").option("compression", "gzip").option("overwriteSchema", "true").\
option("path", path_athena).saveAsTable(tabela)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_tabela: s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_tipos_credito